In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from matplotlib import pyplot as plt
%matplotlib inline
# преобразование строчек словарей в словарь
import ast
import json
# преобразование адресов скринов в md5
import hashlib
# для текстов
# import artm
import os
# для картинок
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.layers import recurrent, BatchNormalization, Activation, Merge, concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate, Bidirectional
# from keras.models import model_from_json

/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [3]:
embeddings = pd.read_csv('embeddings.csv', header=None)
data = pd.read_table("df_export_all_new.tsv")

In [6]:
data.loc[(data['autobanned']==True), 'byHandModerationStatus'] = 'BAD_FOR_RECOMMENDER'
data.loc[(data['tolokaModerationStatus']=='BAD_FOR_RECOMMENDER') & (data['byHandModerationStatus']!='OK'), 'byHandModerationStatus'] = 'BAD_FOR_RECOMMENDER'
data.loc[(data['isSuper']==True) & (data['byHandModerationStatus']=='BAD_FOR_RECOMMENDER'), 'isSuper'] = False
data['res'] = pd.Series(np.zeros(len(data)))
data.loc[data.byHandModerationStatus=='OK', 'res'] = 3
data.loc[data.isSuper==True, 'res'] = 2
data.loc[data.byHandModerationStatus=='BAD_FOR_RECOMMENDER', 'res'] = 1

In [7]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [8]:
big_dict = {}
d_keys = list(ast.literal_eval(data.type_counts[0]).keys())
for key in d_keys:
    big_dict[key] = []
for str_dict in data.type_counts:
    curr_dict = ast.literal_eval(str_dict)
    for key in d_keys:
        big_dict[key].append(curr_dict[key])
big_dict['text_num'] = big_dict['text']
big_dict['text_title'] = big_dict['text-title']
big_dict.pop('text', None)
_ = big_dict.pop('text-title', None)
text_type_features = pd.DataFrame(big_dict)
data = pd.concat([data, text_type_features], axis=1)
train_df = data[data.res!=0]

In [9]:
len(train_df)

8385

In [10]:
X_lstm_train = np.load('X_lstm_train.npy')
y_lstm_train = np.load('y_lstm_train.npy')

In [11]:
X_lstm_train.shape

(8385, 12, 96)

In [12]:
X_lstm = np.zeros((max(train_df.index)+1, max(train_df.slidesCount), 96), dtype=np.float)
y_lstm = np.zeros((max(train_df.index)+1, 3), dtype=np.float)

In [13]:
X_lstm[train_df.index] = X_lstm_train
y_lstm[train_df.index] = y_lstm_train

In [14]:
topic_exp = pd.read_table('topic_probs_expanded.csv')
topic_exp.index = train_df.index
train_df = pd.concat([train_df, topic_exp], axis=1)
train_df.loc[train_df.text.isnull(), topic_exp.columns] = 0.0
tmc = topic_exp.columns[:-3]

In [20]:
topic_LDA_medians = []
for i in range(1,4):
    topic_LDA_medians.append(np.median(df_train.loc[y_train==i, tmc], axis=0))
topic_LDA_means = []
for i in range(1,4):
    topic_LDA_means.append(np.mean(df_train.loc[y_train==i, tmc], axis=0))
topic_LDA_75_quantile = []
for i in range(1,4):
    topic_LDA_75_quantile.append(np.percentile(df_train.loc[y_train==i, tmc], 75, axis=0))
topic_LDA_25_quantile = []
for i in range(1,4):
    topic_LDA_25_quantile.append(np.percentile(df_train.loc[y_train==i, tmc], 25, axis=0))

In [30]:
train_df.res = train_df.res.astype(int)
from scipy.spatial.distance import cosine
def cos_dist_from_median(val):
    res = cosine(topic_LDA_medians[val.res-1], val[tmc.tolist()].tolist())
    if (np.isnan(res)):
        return -1
    return res

def cos_dist_from_mean(val):
    res = cosine(topic_LDA_means[val.res-1], val[tmc.tolist()].tolist())
    if (np.isnan(res)):
        return -1
    return res

def cos_dist_from_75_per(val):
    res = cosine(topic_LDA_75_quantile[val.res-1], val[tmc.tolist()].tolist())
    if (np.isnan(res)):
        return -1
    return res

def cos_dist_from_25_per(val):
    res = cosine(topic_LDA_25_quantile[val.res-1], val[tmc.tolist()].tolist())
    if (np.isnan(res)):
        return -1
    return res

In [31]:
cos_median_distances = train_df.apply(cos_dist_from_median, axis=1)
cos_mean_distances = train_df.apply(cos_dist_from_mean, axis=1)
cos_75_percentile_distaces = train_df.apply(cos_dist_from_75_per, axis=1)
cos_25_percentile_distaces = train_df.apply(cos_dist_from_25_per, axis=1)
train_df['cos_median_distances2'] = cos_median_distances
train_df['cos_mean_distances2'] = cos_mean_distances
train_df['cos_75_percentile_distances'] = cos_75_percentile_distaces
train_df['cos_25_percentile_distances'] = cos_25_percentile_distaces

/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [32]:
y = train_df.res.astype(int)
df = train_df.drop('res', axis=1)

In [33]:
df_train, df_valid, y_train, y_valid = train_test_split(df, y, test_size=0.2, random_state=1, stratify=y)

In [34]:
all_features = ['slidesCount', 
                'publisherFavouritesCount', 'publisherMonetizationAvailable',
               'publisherPublications', 'publisherQualityNumeric',
               'image', 'text_num', 'text_title', 'url', 'video',
                              'topic_0', 'topic_1', 'topic_2',
       'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8',
       'topic_9', 'topic_10', 'topic_11', 'topic_12', 'topic_13', 'topic_14',
       'top_theme', 'cos_median_distances2', 'cos_mean_distances2', 'cos_75_percentile_distances', 'cos_25_percentile_distances']

In [35]:
X_lstm_train = X_lstm[df_train.index]
y_lstm_train = y_lstm[df_train.index]
X_lstm_valid = X_lstm[df_valid.index]
y_lstm_valid = y_lstm[df_valid.index]

In [36]:
binary_y_lstm = np.zeros((len(y_lstm), 2), dtype=np.float)
for i in range(len(y_lstm_train)):
    if ((y_lstm_train[i][0]==1.0)):
        binary_y_lstm[i][0] = 1.0
    else:
        binary_y_lstm[i][1] = 1.0
binary_y_lstm_train = binary_y_lstm[df_train.index]
binary_y_lstm_valid = binary_y_lstm[df_valid.index]

In [37]:
df_train = ss.fit_transform(df_train.loc[:, all_features])
df_valid = ss.transform(df_valid.loc[:, all_features])

iterate bidirectional LSTM

In [38]:
HIDDEN_SIZE = 100
i = 10
lstm_input = Input(shape=(max(train_df.slidesCount), 96,))
lstm_model = Bidirectional(LSTM(HIDDEN_SIZE, recurrent_dropout=0.05, dropout=0.05))(lstm_input)
extra_input = Input(shape=(len(all_features),))
merged = concatenate([lstm_model, extra_input])
prev_layer = merged
for size in range(i):
    lstm_dense = Dense(int(HIDDEN_SIZE/(i+1)), activation='elu')(prev_layer)
    lstm_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(lstm_dense)
    prev_layer = lstm_bn
model_dense = Dense(3, activation='softmax')(prev_layer)
model = Model(inputs=[lstm_input, extra_input], outputs=model_dense)
opt = keras.optimizers.adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile( loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'] )
model_checkpoint = ModelCheckpoint( os.path.join('./','slides_lstm'+str(i)+'.model'), monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
early_stopping = EarlyStopping( monitor='val_acc', patience=10, verbose=1, mode='auto')

history = model.fit([ X_lstm_train, df_train],  y_lstm_train, validation_data=([X_lstm_valid, df_valid], y_lstm_valid), 
                    batch_size=50, epochs=100, callbacks=[model_checkpoint, early_stopping] )

model.load_weights(os.path.join('./','slides_lstm'+str(i)+'.model'))
probs = model.predict([X_lstm_valid, df_valid])
#     y_pred = []
#     for prob in probs:
#         y_pred.append(np.argmax(prob)+1)
#     with open('res_with_themes_2.txt', 'a') as f:
#         f.write(str(i)+'\n')
#         f.write(str(+'\n')
#         f.write(str(classification_report(y_valid, y_pred))+'\n')

/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)`
  # Remove the CWD from sys.path while we load stuff.


Train on 6708 samples, validate on 1677 samples
Epoch 1/100
6708/6708 [==============================] - 24s 4ms/step - loss: 0.9059 - acc: 0.6124 - val_loss: 0.7307 - val_acc: 0.7084
Epoch 2/100
6708/6708 [==============================] - 6s 895us/step - loss: 0.6818 - acc: 0.7095 - val_loss: 0.6352 - val_acc: 0.7460
Epoch 3/100
6708/6708 [==============================] - 6s 889us/step - loss: 0.6182 - acc: 0.7388 - val_loss: 0.6070 - val_acc: 0.7573
Epoch 4/100
6708/6708 [==============================] - 6s 924us/step - loss: 0.5899 - acc: 0.7451 - val_loss: 0.5862 - val_acc: 0.7686
Epoch 5/100
6708/6708 [==============================] - 6s 921us/step - loss: 0.5538 - acc: 0.7660 - val_loss: 0.5582 - val_acc: 0.7835
Epoch 6/100
6708/6708 [==============================] - 6s 924us/step - loss: 0.4840 - acc: 0.8007 - val_loss: 0.4934 - val_acc: 0.7990
Epoch 7/100
6708/6708 [==============================] - 7s 983us/step - loss: 0.4205 - acc: 0.8295 - val_loss: 0.4486 - val_acc: 0

In [40]:
y_pred = []
for prob in probs:
    y_pred.append(np.argmax(prob)+1)
accuracy_score(y_valid, y_pred)

0.92486583184257598

bidirection LSTM

In [54]:
HIDDEN_SIZE = 200
lstm_input = Input(shape=(max(train_df.slidesCount), 96,))
lstm_model = Bidirectional(LSTM(HIDDEN_SIZE, recurrent_dropout=0.05, dropout=0.05))(lstm_input)
lstm_first_dense = Dense(int(HIDDEN_SIZE), activation='elu')(lstm_model)
lstm_first_dropout = Dropout(0.1)(lstm_first_dense)
lstm_first_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(lstm_first_dropout)
lstm_second_dense = Dense(int(HIDDEN_SIZE), activation='elu')(lstm_first_bn)
lstm_second_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(lstm_second_dense)
lstm_third_dense = Dense(int(HIDDEN_SIZE), activation='elu')(lstm_second_bn)
lstm_third_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(lstm_third_dense)
lstm_classify_dense = Dense(3, activation='softmax')(lstm_third_bn)
extra_input = Input(shape=(len(all_features),))
# extra_dense = Dense(len(all_features), activation='linear')(extra_input)
merged = concatenate([lstm_classify_dense, extra_input])
model_first_dense = Dense(7, activation='elu')(merged)
moden_first_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(model_first_dense)
model_dense = Dense(3, activation='softmax')(moden_first_bn)
model = Model(inputs=[lstm_input, extra_input], outputs=model_dense)
opt = keras.optimizers.adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile( loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'] )
model_checkpoint = ModelCheckpoint( os.path.join('./','slides_lstm.model'), monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
early_stopping = EarlyStopping( monitor='val_acc', patience=10, verbose=1, mode='auto')

history = model.fit([ X_lstm_train, df_train],  y_lstm_train, validation_data=([X_lstm_valid, df_valid], y_lstm_valid), 
                    batch_size=50, epochs=100, callbacks=[model_checkpoint, early_stopping] )

/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  # Remove the CWD from sys.path while we load stuff.
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  app.launch_new_instance()


Train on 6708 samples, validate on 1677 samples
Epoch 1/100
6708/6708 [==============================] - 13s 2ms/step - loss: 0.9583 - acc: 0.5705 - val_loss: 0.7708 - val_acc: 0.6834
Epoch 2/100
6708/6708 [==============================] - 8s 1ms/step - loss: 0.7354 - acc: 0.6999 - val_loss: 0.7054 - val_acc: 0.7120
Epoch 3/100
6708/6708 [==============================] - 8s 1ms/step - loss: 0.6941 - acc: 0.7056 - val_loss: 0.6799 - val_acc: 0.7174
Epoch 4/100
6708/6708 [==============================] - 8s 1ms/step - loss: 0.6789 - acc: 0.7106 - val_loss: 0.6647 - val_acc: 0.7299
Epoch 5/100
6708/6708 [==============================] - 7s 1ms/step - loss: 0.6726 - acc: 0.7080 - val_loss: 0.6779 - val_acc: 0.7203
Epoch 6/100
6708/6708 [==============================] - 8s 1ms/step - loss: 0.6678 - acc: 0.7123 - val_loss: 0.6644 - val_acc: 0.7287
Epoch 7/100
6708/6708 [==============================] - 8s 1ms/step - loss: 0.6697 - acc: 0.7145 - val_loss: 0.6682 - val_acc: 0.7233
Epoch 

In [40]:
np.set_printoptions(threshold=np.nan)
# print(df_train.index.values)

In [50]:
HIDDEN_SIZE = 200
lstm_input = Input(shape=(max(train_df.slidesCount), 96,))
lstm_model = LSTM(HIDDEN_SIZE, recurrent_dropout=0.05, dropout=0.05)(lstm_input)
lstm_first_dense = Dense(int(HIDDEN_SIZE/4), activation='elu')(lstm_model)
lstm_first_dropout = Dropout(0.1)(lstm_first_dense)
lstm_first_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(lstm_first_dropout)
lstm_second_dense = Dense(int(HIDDEN_SIZE/5), activation='elu')(lstm_first_bn)
lstm_second_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(lstm_second_dense)
lstm_third_dense = Dense(int(HIDDEN_SIZE/6), activation='elu')(lstm_second_bn)
lstm_third_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(lstm_third_dense)
lstm_classify_dense = Dense(3, activation='softmax')(lstm_third_bn)
extra_input = Input(shape=(len(all_features),))
# extra_dense = Dense(len(all_features), activation='linear')(extra_input)
merged = concatenate([lstm_classify_dense, extra_input])
model_first_dense = Dense(7, activation='elu')(merged)
moden_first_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(model_first_dense)
model_dense = Dense(3, activation='softmax')(moden_first_bn)
model = Model(inputs=[lstm_input, extra_input], outputs=model_dense)
opt = keras.optimizers.adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile( loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'] )
model_checkpoint = ModelCheckpoint( os.path.join('./','slides_lstm.model'), monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
early_stopping = EarlyStopping( monitor='val_acc', patience=10, verbose=1, mode='auto')

history = model.fit([ X_lstm_train, df_train],  y_lstm_train, validation_data=([X_lstm_valid, df_valid], y_lstm_valid), 
                    batch_size=50, epochs=100, callbacks=[model_checkpoint, early_stopping] )

/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  # Remove the CWD from sys.path while we load stuff.
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  app.launch_new_instance()


Train on 6708 samples, validate on 1677 samples
Epoch 1/100
6708/6708 [==============================] - 7s 1ms/step - loss: 0.9317 - acc: 0.5784 - val_loss: 0.7469 - val_acc: 0.7245
Epoch 2/100
6708/6708 [==============================] - 5s 800us/step - loss: 0.7212 - acc: 0.7024 - val_loss: 0.6929 - val_acc: 0.7209
Epoch 3/100
6708/6708 [==============================] - 5s 813us/step - loss: 0.6758 - acc: 0.7162 - val_loss: 0.7021 - val_acc: 0.7001
Epoch 4/100
6708/6708 [==============================] - 5s 809us/step - loss: 0.6490 - acc: 0.7182 - val_loss: 0.6774 - val_acc: 0.7346
Epoch 5/100
6708/6708 [==============================] - 5s 747us/step - loss: 0.6363 - acc: 0.7291 - val_loss: 0.6369 - val_acc: 0.7376
Epoch 6/100
6708/6708 [==============================] - 5s 814us/step - loss: 0.6293 - acc: 0.7311 - val_loss: 0.6344 - val_acc: 0.7305
Epoch 7/100
6708/6708 [==============================] - 6s 823us/step - loss: 0.6242 - acc: 0.7317 - val_loss: 0.6309 - val_acc: 0.

In [56]:
model.load_weights(os.path.join('./','slides_lstm.model'))

In [57]:
probs = model.predict([X_lstm_valid, df_valid])

In [58]:
y_pred = []
for prob in probs:
    y_pred.append(np.argmax(prob)+1)

In [59]:
accuracy_score(y_valid, y_pred)

0.76147883124627314

In [26]:
print(classification_report(y_valid, y_pred))

             precision    recall  f1-score   support

          1       0.73      0.47      0.57       204
          2       0.63      0.41      0.50       359
          3       0.76      0.90      0.83      1114

avg / total       0.73      0.74      0.72      1677



In [41]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [131]:
model = Sequential()
HIDDEN_SIZE = 200
model.add(Conv1D(filters=32, kernel_size=3, padding='same', input_shape=(max(train_df.slidesCount), 96), activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(recurrent.LSTM(HIDDEN_SIZE,  recurrent_dropout=0.1, dropout=0.2))
# model.add( Dense( int(HIDDEN_SIZE/5), activation='relu' ))
model.add( Dropout(0.1) )
model.add( Dense( 2, activation='sigmoid' ) )
opt = keras.optimizers.Nadam(lr=0.002, epsilon=1e-09)
model.compile( loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'] )

model_checkpoint = ModelCheckpoint( os.path.join('./','slides_lstm.model'), monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
early_stopping = EarlyStopping( monitor='val_acc', patience=10, verbose=1, mode='auto')

history = model.fit( X_lstm_train, binary_y_lstm_train, validation_data=(X_lstm_valid, binary_y_lstm_valid),
                    batch_size=30, epochs=100, callbacks=[model_checkpoint, early_stopping] )

Train on 12028 samples, validate on 1337 samples
Epoch 1/100
12028/12028 [==============================] - 10s 842us/step - loss: 0.3470 - acc: 0.8833 - val_loss: 0.3039 - val_acc: 0.8994
Epoch 2/100
12028/12028 [==============================] - 7s 546us/step - loss: 0.3338 - acc: 0.8849 - val_loss: 0.3041 - val_acc: 0.9009
Epoch 3/100
12028/12028 [==============================] - 7s 541us/step - loss: 0.3311 - acc: 0.8867 - val_loss: 0.3023 - val_acc: 0.9013
Epoch 4/100
12028/12028 [==============================] - 6s 534us/step - loss: 0.3265 - acc: 0.8872 - val_loss: 0.3000 - val_acc: 0.9009
Epoch 5/100
12028/12028 [==============================] - 6s 535us/step - loss: 0.3229 - acc: 0.8877 - val_loss: 0.3023 - val_acc: 0.9028
Epoch 6/100
12028/12028 [==============================] - 6s 529us/step - loss: 0.3155 - acc: 0.8893 - val_loss: 0.3035 - val_acc: 0.8998
Epoch 7/100
12028/12028 [==============================] - 6s 535us/step - loss: 0.3087 - acc: 0.8923 - val_loss: 0.

In [132]:
y_other_valid = []
for y_val in y_valid:
    if (y_val==1.0):
        y_other_valid.append(0)
    else:
        y_other_valid.append(1)

In [133]:
probs_bad = model.predict_proba(X_lstm_valid)
y_pred_bad = []
for prob in probs_bad:
    y_pred_bad.append(np.argmax(prob))
print(classification_report(y_other_valid, y_pred_bad))
# 0 - это плохо; 1 - остальные

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        96
          1       0.93      0.98      0.95      1241

avg / total       0.86      0.91      0.88      1337



In [79]:
y_other_valid = []
for y_val in y_valid:
    if (y_val==2.0):
        y_other_valid.append(0)
    else:
        y_other_valid.append(1)

In [83]:
probs_super = model.predict_proba(X_lstm_valid)
y_pred_super = []
for prob in probs_super:
    y_pred_super.append(np.argmax(prob))
print(classification_report(y_other_valid, y_pred_super))
# 0 - это супер; 1 - остальные

             precision    recall  f1-score   support

          0       0.15      0.15      0.15       179
          1       0.77      0.78      0.77       660

avg / total       0.64      0.64      0.64       839



In [204]:
X_lstm_train.shape

(12696, 12, 96)

In [203]:
df_train.shape

(12696, 10)

In [234]:
HIDDEN_SIZE = 200
lstm_input = Input(shape=(max(train_df.slidesCount), 96,))
lstm_model = LSTM(HIDDEN_SIZE, recurrent_dropout=0.05, dropout=0.05)(lstm_input)
lstm_first_dense = Dense(int(HIDDEN_SIZE/4), activation='elu')(lstm_model)
# lstm_first_dropout = Dropout(0.1)(lstm_first_dense)
lstm_first_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(lstm_first_dense)
lstm_second_dense = Dense(int(HIDDEN_SIZE/5), activation='elu')(lstm_first_bn)
lstm_second_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(lstm_second_dense)
lstm_third_dense = Dense(int(HIDDEN_SIZE/6), activation='elu')(lstm_second_bn)
lstm_third_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(lstm_third_dense)
lstm_classify_dense = Dense(3, activation='softmax')(lstm_third_bn)
extra_input = Input(shape=(len(all_features),))
# extra_dense = Dense(len(all_features), activation='linear')(extra_input)
merged = concatenate([lstm_classify_dense, extra_input])
model_first_dense = Dense(7, activation='elu')(merged)
moden_first_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(model_first_dense)
model_dense = Dense(3, activation='softmax')(moden_first_bn)
model = Model(inputs=[lstm_input, extra_input], outputs=model_dense)
opt = keras.optimizers.adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile( loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'] )
model_checkpoint = ModelCheckpoint( os.path.join('./','slides_lstm.model'), monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
early_stopping = EarlyStopping( monitor='val_acc', patience=10, verbose=1, mode='auto')

history = model.fit([ X_lstm_train, df_train],  y_lstm_train, validation_data=([X_lstm_valid, df_valid], y_lstm_valid), 
                    batch_size=50, epochs=100, callbacks=[model_checkpoint, early_stopping] )

/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  """
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  import sys
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  if __name__ == '__main__':
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  from ipykernel import kernelapp as app


Train on 12696 samples, validate on 669 samples
Epoch 1/100
12696/12696 [==============================] - 15s 1ms/step - loss: 0.7699 - acc: 0.7213 - val_loss: 0.5341 - val_acc: 0.8012
Epoch 2/100
12696/12696 [==============================] - 6s 487us/step - loss: 0.5389 - acc: 0.7932 - val_loss: 0.4940 - val_acc: 0.8102
Epoch 3/100
12696/12696 [==============================] - 6s 481us/step - loss: 0.5227 - acc: 0.7976 - val_loss: 0.4786 - val_acc: 0.8072
Epoch 4/100
12696/12696 [==============================] - 6s 489us/step - loss: 0.5124 - acc: 0.8009 - val_loss: 0.4677 - val_acc: 0.8161
Epoch 5/100
12696/12696 [==============================] - 6s 481us/step - loss: 0.5049 - acc: 0.8007 - val_loss: 0.4708 - val_acc: 0.7922
Epoch 6/100
12696/12696 [==============================] - 6s 484us/step - loss: 0.5010 - acc: 0.8027 - val_loss: 0.4667 - val_acc: 0.8117
Epoch 7/100
12696/12696 [==============================] - 6s 474us/step - loss: 0.4915 - acc: 0.8062 - val_loss: 0.456

In [235]:
HIDDEN_SIZE = 200
lstm_input = Input(shape=(max(train_df.slidesCount), 96,))
lstm_model = LSTM(HIDDEN_SIZE, recurrent_dropout=0.05, dropout=0.1)(lstm_input)
lstm_first_dense = Dense(int(HIDDEN_SIZE/4), activation='elu')(lstm_model)
# lstm_first_dropout = Dropout(0.1)(lstm_first_dense)
lstm_first_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(lstm_first_dense)
lstm_second_dense = Dense(int(HIDDEN_SIZE/5), activation='elu')(lstm_first_bn)
lstm_second_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(lstm_second_dense)
lstm_third_dense = Dense(int(HIDDEN_SIZE/6), activation='elu')(lstm_second_bn)
lstm_third_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(lstm_third_dense)
lstm_classify_dense = Dense(3, activation='softmax')(lstm_third_bn)
extra_input = Input(shape=(len(all_features),))
# extra_dense = Dense(len(all_features), activation='linear')(extra_input)
merged = concatenate([lstm_classify_dense, extra_input])
model_first_dense = Dense(7, activation='elu')(merged)
moden_first_bn = BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)(model_first_dense)
model_dense = Dense(3, activation='softmax')(moden_first_bn)
model = Model(inputs=[lstm_input, extra_input], outputs=model_dense)
opt = keras.optimizers.adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile( loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'] )
model_checkpoint = ModelCheckpoint( os.path.join('./','slides_lstm.model'), monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
early_stopping = EarlyStopping( monitor='val_acc', patience=10, verbose=1, mode='auto')

history = model.fit([ X_lstm_train, df_train],  y_lstm_train, validation_data=([X_lstm_valid, df_valid], y_lstm_valid), 
                    batch_size=50, epochs=100, callbacks=[model_checkpoint, early_stopping] )

/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  # Remove the CWD from sys.path while we load stuff.
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  app.launch_new_instance()


Train on 12696 samples, validate on 669 samples
Epoch 1/100
12696/12696 [==============================] - 15s 1ms/step - loss: 0.8401 - acc: 0.6755 - val_loss: 0.5483 - val_acc: 0.8042
Epoch 2/100
12696/12696 [==============================] - 6s 505us/step - loss: 0.5501 - acc: 0.7917 - val_loss: 0.4936 - val_acc: 0.8132
Epoch 3/100
12696/12696 [==============================] - 6s 478us/step - loss: 0.5178 - acc: 0.7977 - val_loss: 0.4644 - val_acc: 0.8072
Epoch 4/100
12696/12696 [==============================] - 6s 483us/step - loss: 0.5093 - acc: 0.8001 - val_loss: 0.4567 - val_acc: 0.8102
Epoch 5/100
12696/12696 [==============================] - 6s 491us/step - loss: 0.5014 - acc: 0.7986 - val_loss: 0.4602 - val_acc: 0.8191
Epoch 6/100
12696/12696 [==============================] - 6s 484us/step - loss: 0.4954 - acc: 0.7999 - val_loss: 0.4438 - val_acc: 0.8176
Epoch 7/100
12696/12696 [==============================] - 6s 484us/step - loss: 0.4924 - acc: 0.8000 - val_loss: 0.443

In [217]:
model = Sequential()
HIDDEN_SIZE = 200
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', input_shape=(max(train_df.slidesCount), 96), activation='elu'))
# model.add(MaxPooling1D(pool_size=2))
model.add(recurrent.LSTM(HIDDEN_SIZE, input_shape=(max(train_df.slidesCount), 96), recurrent_dropout=0.05, dropout=0.05))
model.add( Dense( int(HIDDEN_SIZE/4), activation='elu' ))
# model.add( Dropout(0.1) )
model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))
model.add( Dense( int(HIDDEN_SIZE/5), activation='elu' ))
model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))
model.add( Dropout(0.1) )
model.add( Dense( int(HIDDEN_SIZE/6), activation='elu' ))
model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))
# model.add( Dropout(0.1) )
model.add( Dense( 3, activation='softmax' ) )

opt = keras.optimizers.adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
model.compile( loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'] )

model_checkpoint = ModelCheckpoint( os.path.join('./','slides_lstm.model'), monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
early_stopping = EarlyStopping( monitor='val_acc', patience=10, verbose=1, mode='auto')

history = model.fit( X_lstm_train, y_lstm_train, validation_data=(X_lstm_valid, y_lstm_valid), 
                    batch_size=50, epochs=100, callbacks=[model_checkpoint, early_stopping] )

(None, 3)
(None, 10)


/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  # Remove the CWD from sys.path while we load stuff.
/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9, weights=None, epsilon=1e-06)`
  del sys.path[0]


ValueError: `Concatenate` layer should be called on a list of inputs

In [236]:
model.load_weights(os.path.join('./','slides_lstm.model'))

In [241]:
probs = model.predict([X_lstm_valid, df_valid])

In [243]:
y_pred = []
for prob in probs:
    y_pred.append(np.argmax(prob)+1)

In [244]:
accuracy_score(y_valid, y_pred)

0.84304932735426008

In [245]:
print(classification_report(y_valid, y_pred))

             precision    recall  f1-score   support

          1       0.62      0.50      0.55        48
          2       0.67      0.48      0.56        90
          3       0.88      0.94      0.91       531

avg / total       0.83      0.84      0.83       669



In [246]:
Y_valid = np.argmax(y_valid, axis=1) # Convert one-hot to index
y_pred = model.predict_classes([X_lstm_valid, df_valid])
print(classification_report(Y_valid, y_pred))

/nfs/home/pgulyaev/diploma/cv/ipython/lib/python3.5/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


AttributeError: 'Model' object has no attribute 'predict_classes'

In [164]:
model.load_weights(os.path.join('./','slides_lstm.model'))

In [165]:
probs = model.predict_proba(X_lstm_valid)

In [171]:
probs = model.predict_proba(X_lstm)

In [174]:
np.save('probs.npy', probs)

In [166]:
y_pred = []
for prob in probs:
    y_pred.append(np.argmax(prob)+1)

In [167]:
accuracy_score(y_valid, y_pred)

0.81763826606875933

In [168]:
print(classification_report(y_valid, y_pred))

             precision    recall  f1-score   support

          1       0.58      0.38      0.46        48
          2       0.70      0.21      0.32        90
          3       0.83      0.96      0.89       531

avg / total       0.80      0.82      0.79       669



In [144]:
probs = model.predict_proba(X_lstm_valid)

In [145]:
y_pred = []
for prob in probs:
    y_pred.append(np.argmax(prob)+1)

In [147]:
accuracy_score(y_valid, y_pred)

0.7808526551982049

In [146]:
print(classification_report(y_valid, y_pred))

             precision    recall  f1-score   support

          1       0.50      0.47      0.48        96
          2       0.40      0.25      0.31       179
          3       0.84      0.90      0.87      1062

avg / total       0.76      0.78      0.77      1337



In [99]:
probs = model.predict_proba(X_lstm_valid)

In [100]:
y_pred = []
for prob in probs:
    y_pred.append(np.argmax(prob)+1)

In [101]:
print(classification_report(y_valid, y_pred))

             precision    recall  f1-score   support

          1       0.55      0.42      0.47        96
          2       0.41      0.42      0.42       179
          3       0.85      0.87      0.86      1062

avg / total       0.77      0.78      0.77      1337



Без дополнительных ок-ов

In [75]:
print(classification_report(y_valid, y_pred))

             precision    recall  f1-score   support

          1       0.71      0.57      0.63       102
          2       0.56      0.44      0.49       179
          3       0.78      0.86      0.81       558

avg / total       0.72      0.73      0.72       839



In [102]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts/sum(counts)))

{1: 0.071380471380471378, 2: 0.13408155630377852, 3: 0.79453797231575007}